# Cleaning Searchlight

## PLEASE READ BELOW!

Thank you so much for helping GoodlyLabs clean Searchlight's Daily Congressional Database! For context, our database consists of all speeches said on the floor by both representatives and senators in the US Congress over the past 24 years. Currently, we manage all this data by storing information about the speakers (the state they represent, the party they belong to, etc.) and information about the speeches (the date it was said, the actual speech text, etc.) in two separate tables. The speakers table is about two thousand rows long, and the speeches table is about 1 million rows long. Unfortunately, with this much data, there's bound to be errors.

The following jupyter notebook exists so that volunteers such as yourselves can easily help us edit/clean both the speakers and speeches table. Each table has its own associated challenges and errors, which we will explain in the sections below. At the end of this notebook, you will have two final products: a cleaned speakers table and a cleaned speeches table in csv format, which you will then zip and upload to this Google Drive folder here (<url>).

# Initialization

Don't worry about the inner workings of these functions or packages, just run all of the cells below (except for the first, which you will have to edit). Later, for each section, we'll explain exactly what each corresponding function does and how you can use it.

For the cell directly below, use the excel spreadsheet here (<url>) to determine what values you specifically need to fill in for the constants below.

In [8]:
SPEECHES_TABLE_START = 0
SPEECHES_TABLE_END = 10000
SPEAKERS_TABLE_START = 0
SPEAKERS_TABLE_END = 100
NOTEBOOK_ID = 1

#If you're curious, what these constants are doing is constraining the dataset to just 
#the small subset you have been assigned to clean. 

In [3]:
import pandas as pd
import re
import numpy as np
import urllib
from pathlib import Path
import math

In [58]:
speakers = pd.read_csv('allspeakers.csv')

In [60]:
# speeches = pd.read_csv('allspeeches.csv')

In [71]:
def fix_district(index, district):
    speakers.loc[index, 'district'] = district
    print(str(speakers.loc[index, 'last_name']) + "'s district is now: " + str(speakers.loc[index, 'district']))

In [72]:
def fix_type(index, member_type):
    speakers.loc[index, 'type'] = member_type
    print(str(speakers.loc[index, 'last_name']) + "'s type is now: " + str(speakers.loc[index, 'type']))

# Cleaning Speakers

## The speakers table currently has only one major error: missing missing district values.

Note: It is certainly possible that your subset of the speakers table has no errors, and so if you see no errors or mistakes when cells are run, don't be alarmed. That just means you have less work to do!

### Fixing District Values

In [90]:
# Run this cell to see which rows in the speakers table (specifically, those who are house reps,
# since senators are not assigned districts) are missing district values.
representatives = speakers[speakers['chamber'] == 'HOUSE']
representatives[representatives['district'].isnull()]

,speaker_id,first_name,last_name,chamber,type,party,state,district,bio_guide_id,congress_id
25,2350.0,Jodey,ARRINGTON,HOUSE,REPRESENTATIVE,R,TX,NaN,A000375,115.0
34,2337.0,Don,BACON,HOUSE,REPRESENTATIVE,R,NE,NaN,B001298,115.0
40,1734.0,Frank,BALLANCE,HOUSE,REPRESENTATIVE,D,NC,NaN,B001238,108.0
42,2326.0,Jim,BANKS,HOUSE,REPRESENTATIVE,R,IN,NaN,B001299,115.0
44,51.0,Peter,BARCA,HOUSE,REPRESENTATIVE,D,WI,NaN,B001226,103.0
47,53.0,Tom,BARLOW,HOUSE,REPRESENTATIVE,D,KY,NaN,B000151,103.0
50,2311.0,Nanette,BARRAGAN,HOUSE,REPRESENTATIVE,D,CA,NaN,B001300,115.0
68,72.0,Anthony,BEILENSON,HOUSE,REPRESENTATIVE,D,CA,NaN,B000318,103.0
74,78.0,Helen,BENTLEY,HOUSE,REPRESENTATIVE,R,MD,NaN,B000392,103.0
79,2333.0,Jack,BERGMAN,HOUSE,REPRESENTATIVE,R,MI,NaN,B001301,115.0


Above, each row represents a congress member. Observe that the table above has a NaN value for each entry in the "district" column. To fix the district values in your subset of the database, you will be using the "fix_district" function. This function takes in two inputs: "index", the index of the row (which can be found in the left most column in bold), and "district", the district of the congressmember in the row. 

For each congressmember above, look up their first and last name in a search engine of your choice, and find which congressional district they represented in their state (1st district, 2nd district, etc). If they belong to a state which only has a single, at-large district, then consider that the 1st district. The value you will enter for the "district" input to fix_district is the integer value of their district. 

So, for example, if the index of the congress member's row who you were trying to fix was 48, and your congress member represented the 33rd district of their state, you would enter: "fix_district(48, 33)" in the cell below. You should see a print message confirming that the senator's district has been changed.

Example print message: "APPLEGATE's district is now: 33.0"

In [66]:
fix_district("fill", "fill")

APPLEGATE's district is now: 33.0


Repeat the process above until there are no more representatives left in your subset of the dataset that are missing district values. You can keep repeatedly running the cell underneath the subheading "Fixing District Values" to check how many representatives' district you have left to update.

# Cleaning Speeches

## The speeches table currently has two major errors: missing speaker_id values and missing proceeding_title values.

Note: It is certainly possible that your subset of the speakers table has no errors, and so if you see no errors or mistakes when cells are run, don't be alarmed. That just means you have less work to do!

In [92]:
speeches.columns

Index(['speech_id', 'topic_id', 'word_count', 'speech_text', 'file_name',
       'mods_file', 'speaker_id', 'proceeding_title', 'year', 'month', 'day'],
      dtype='object')

In [109]:
speeches[speeches['speaker_id'].isnull()]

,speech_id,topic_id,word_count,speech_text,file_name,mods_file,speaker_id,proceeding_title,year,month,day
36,36,topic_id,95,"Mr.President, on February 2, 1994, the Committ...",CREC-1994-03-02-pt1-PgS56.txt,CREC-1994-03-02-pt1-PgS56.xml,NaN,NaN,1994,March,2
40,40,topic_id,266,"Mr.President, a group of us from the leadershi...",CREC-1994-02-25-pt1-PgH14.txt,CREC-1994-02-25-pt1-PgH14.xml,NaN,NaN,1994,February,25
42,42,topic_id,198,I thank the distinguished manager. Let me rep...,CREC-1994-02-25-pt1-PgH14.txt,CREC-1994-02-25-pt1-PgH14.xml,NaN,NaN,1994,February,25
319,319,topic_id,8,"Mr.President, will the Senator yield for a que...",CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
321,321,topic_id,238,"Mr.President, the Senator from Illinois, my di...",CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
323,323,topic_id,10,You will be telling how old we are. Be careful.,CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
325,325,topic_id,107,This is a stand-alone resolution. I have never...,CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
328,328,topic_id,3717,"Mr.President, I thank the Senator from Nevada ...",CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
331,331,topic_id,39,"I suggest the absence of a quorum, with the ti...",CREC-1994-02-25-pt1-PgH15.txt,CREC-1994-02-25-pt1-PgH15.xml,NaN,NaN,1994,February,25
682,682,topic_id,48,"Mr.President, just one parliamentary inquiry. ...",CREC-1994-01-27-pt1-PgH4.txt,CREC-1994-01-27-pt1-PgH4.xml,NaN,NaN,1994,January,27
